In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import skimage.transform
from sklearn.model_selection import train_test_split
from network import abstract_network
import cv2

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def read_pictures(directory='train/'):
    with open('files_list.txt') as f:
        files_list = f.readlines()
    files_list = [(directory+i.strip()) for i in files_list]
    return files_list

In [3]:
input_shape = (256, 256, 3)
files_list = read_pictures('resized_train/')

In [4]:
# def preprocess_img(img):
#     return skimage.transform.resize(img, input_shape)

In [5]:
# def get_dataset(preprocess = True, files_num=1000, random=False):
#     X = np.empty((files_num, *input_shape), dtype=np.float32)
#     y = np.empty((files_num, 2), dtype=np.uint8)
    
#     if random:
#         files_to_read = np.random.choice(files_list, files_num)
#     else:
#         files_to_read = files_list[:files_num]
        
#     if preprocess:
#         operation = lambda x: preprocess_img(cv2.imread(x))[:,:,::-1]
#     else:
#         operation = lambda x: cv2.imread(x)[:,:,::-1]
        
#     for num, i in enumerate(tqdm.tqdm(files_to_read)):
#         X[num] = operation(i)
            
#         if 'cat' in i:
#             y[num] = [1, 0]
#         else:
#             y[num] = [0, 1]
#     return X, y

In [6]:
# def save_preprocessed_images(images):
#     for i, name in tqdm.tqdm(zip(images, files_list)):
#         plt.imsave(f'preprocessed_images/{name.split("/")[-1]}', i)

In [7]:
# X, y = get_dataset(preprocess=True, files_num=25000, random=False)
# X = X/np.max(X)

In [8]:
# save_preprocessed_images(X)

In [9]:
X_train, X_test = train_test_split(files_list, test_size=0.3)

# Free memory
# del X
# del y

In [10]:
# plt.figure(figsize=(10,10))
# plt.subplot(121)
# plt.imshow(X_train[1]);
# plt.subplot(122)
# plt.imshow(X_train[0])
# plt.show()

In [12]:
class cnn_network(abstract_network):       
    def _inference(self):
        inp = self.input
        with tf.name_scope('classificator'):
            with tf.name_scope('convolution'):
                layer = tf.layers.conv2d(inp, 64, 3, activation=tf.nn.relu)
                layer = tf.layers.max_pooling2d(layer, pool_size=3, strides=2)
        
                layer = tf.layers.conv2d(layer, 64, 3, activation=tf.nn.relu)
                layer = tf.layers.max_pooling2d(layer, pool_size=3, strides=2)
                layer = tf.nn.dropout(layer, self.dropout_prob)
                
                layer = tf.layers.conv2d(layer, 64, 3, activation=tf.nn.relu)
                layer = tf.layers.max_pooling2d(layer, pool_size=3, strides=2)
                
                layer = tf.layers.conv2d(layer, 64, 3, activation=tf.nn.relu, padding='same')
                layer = tf.layers.max_pooling2d(layer, pool_size=3, strides=2)
                
                layer = tf.layers.conv2d(layer, 64, 3, activation=tf.nn.relu, padding='same')
                layer = tf.layers.max_pooling2d(layer, pool_size=3, strides=2)
                layer = tf.nn.dropout(layer, self.dropout_prob)
                
#                 layer = tf.layers.conv2d(layer, 16, , activation=tf.nn.relu)
#                 layer = tf.layers.max_pooling2d(layer, pool_size=3, strides=2)
                
                layer = tf.layers.flatten(layer)

            with tf.name_scope('dense'):
                layer = tf.layers.dense(layer, 1024, activation=tf.nn.relu)
                layer = tf.nn.dropout(layer, self.dropout_prob)
                layer = tf.layers.dense(layer, 512, activation=tf.nn.relu)
                layer = tf.layers.batch_normalization(layer)
                layer = tf.layers.dense(layer, 256, activation=tf.nn.relu)
                layer = tf.layers.dense(layer, 128, activation=tf.nn.relu)
#                 layer = tf.layers.batch_normalization(layer)
                layer = tf.layers.dense(layer, 2, activation=tf.nn.softmax)

        return layer

In [13]:
tf.reset_default_graph()
network = cnn_network(input_shape)
network.set_training_data(X_train, X_test)
network.training(batch_size=128, epochs=15, iter_before_validation=1)

50% of training is class 0
49% of test is class 0


100%|████████████████████████████████████████████████████████████████████████████████| 274/274 [00:43<00:00,  6.33it/s]


Test accuracy: 61% 	Training loss: 0.6900765608697045 	 Training accuracy: 52%


100%|████████████████████████████████████████████████████████████████████████████████| 274/274 [00:40<00:00,  6.70it/s]


Test accuracy: 63% 	Training loss: 0.6272738519376214 	 Training accuracy: 64%


...

Test accuracy: 86% 	Training loss: 0.4412205737002575 	 Training accuracy: 86%


100%|████████████████████████████████████████████████████████████████████████████████| 274/274 [00:40<00:00,  6.73it/s]


Test accuracy: 87% 	Training loss: 0.4362156576048718 	 Training accuracy: 87%


100%|████████████████████████████████████████████████████████████████████████████████| 274/274 [00:40<00:00,  6.74it/s]


Test accuracy: 88% 	Training loss: 0.4362049980537735 	 Training accuracy: 86%


In [14]:
assert len([op.name for op in tf.get_default_graph().get_operations() if 'output' in op.name or 'features' in op.name]) == 2
assert len([op.name for op in tf.get_default_graph().get_operations() if 'cnn_' in op.name]) == 0

In [16]:
network.save_model('saved_model/classifier_256x256_full.ckpt')